In [ ]:
import OCC
import meshplot as mp
import numpy as np
import igl
import k3d
from geomdl import BSpline
import glob
import os
from yaml import CLoader
import yaml

In [20]:
weights = len(list(filter(lambda x: x > 0, [0.1, 0.5, 1.0, 2.0])))
weights = [True, True, False, True, False]
print(sum(weights))

3


In [ ]:
# Statistics
from tqdm.auto import tqdm
files = glob.glob("res/*.yml")
curve2d_weights = []
curve2d_rational = []
curve2d_closed = []

curve3d_weights = []
curve3d_rational = []
curve3d_closed = []

surface_weights = []
surface_rational = []
surface_closed = []
surface_trimmed = []
curve2d_total = 0
curve3d_total = 0
surface_total = 0
print(len(files))
for f in tqdm(files[:]):
    with open(f, "r") as fi:
        d = yaml.load(fi, Loader=CLoader)
        
        surface_total += len(d["surfaces"])
        for c in d["surfaces"]:
            surface_weights.append(len(list(filter(lambda x: x[0] < 0, c["weights"]))) == 0)
            surface_weights.append(len(list(filter(lambda x: x[1] < 0, c["weights"]))) == 0)
            surface_rational.append(c["u_rational"])
            surface_rational.append(c["v_rational"])
            surface_closed.append(c["u_closed"])
            surface_closed.append(c["v_closed"])
            surface_trimmed.append(c["is_trimmed"])
        
        curve3d_total += len(d["curves"])
        for c in d["curves"]:
            curve3d_weights.append(len(list(filter(lambda x: x < 0, c["weights"]))) == 0)
            curve3d_rational.append(c["rational"])
            curve3d_closed.append(c["closed"])
        
        curve2d_total += len(d["trim"])
        for c in d["trim"]:
            curve2d_weights.append(len(list(filter(lambda x: x < 0, c["weights"]))) == 0)
            curve2d_rational.append(c["rational"])
            curve2d_closed.append(c["closed"])
            
print("3D Curves: Rational: %0.2f, Closed: %0.2f, Pos_Weights: %0.2f"%(sum(curve3d_rational)/curve3d_total*100, sum(curve3d_closed)/curve3d_total*100, sum(curve3d_weights)/curve3d_total*100))
print("2D Curves: Rational: %0.2f, Closed: %0.2f, Pos_Weights: %0.2f"%(sum(curve2d_rational)/curve2d_total*100, sum(curve2d_closed)/curve2d_total*100, sum(curve2d_weights)/curve2d_total*100))
print("Surfaces : Rational: %0.2f, Closed: %0.2f, Pos_Weights: %0.2f, Trimmed: %0.2f"%(sum(surface_rational)/surface_total*50, sum(surface_closed)/surface_total*50, sum(surface_weights)/surface_total*50, sum(surface_trimmed)/surface_total*100))

In [25]:
3D Curves Rational: 28.95%, Closed: 3.62%, Positive Weights: 100.0%
2D Curves Rational: 5.75%, Closed: 1.21%, Positive Weights: 100.0%
Surfaces Rational: 37.74%, Closed: 3.73%, Positive Weights: 100.0%, Trimmed: 61.03%

SyntaxError: invalid syntax (<ipython-input-25-2ac1d4dfb54d>, line 1)

In [27]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone, IFSelect_ItemsByEntity
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_NurbsConvert
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Extend.TopologyUtils import TopologyExplorer, WireExplorer
from OCCUtils.Topology import Topo, dumpTopology
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.BRep import BRep_Tool
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Pnt2d

In [79]:
step_files = sorted(glob.glob("data/*.step"))
for s in step_files[1:2]:
    
    #print(s)
    shapes = read_step_file(s)
    print(s, shapes)
    
    # Iterating over parts
    for part in shapes[:]:
        topo, mesh = process_part(part)
        
        # Iterating over solids
        for solid_idx, solid in enumerate(topo.solids()):
            process_solid(solid_idx, solid)

            # Iterating over shells
            for shell_idx, shell in enumerate(topo.shells_from_solids(solid)):
                process_shell(shell_idx, shell)

                # Iterating over faces
                vertices = []
                faces = []
                mesh_normals = []
                surface_normals = []
                face_centroids = []
                colors = []
                for face_idx, face in enumerate(topo.faces_from_shells(shell)):
                    verts, tris, mesh_norms, centroids, surf_norms = process_face(face_idx, face, first_vertex=len(vertices))
                    colors.extend([face_idx]*len(tris))
                    vertices.extend(verts)
                    faces.extend(tris)
                    mesh_normals.extend(mesh_norms)
                    surface_normals.extend(surf_norms)
                    face_centroids.extend(centroids)
                    
                    #print(face_idx)
                    #if face_idx == 5:
                    #    break
                
                vertices = np.array(vertices)
                faces = np.array(faces)
                mesh_normals = np.array(mesh_normals)
                surface_normals = np.array(surface_normals)
                face_centroids = np.array(face_centroids)
                colors = np.array(colors)
                
                sf = (np.max(vertices) - np.min(vertices)) / 50.0
                #print(vertices.shape, faces.shape, mesh_normals.shape, surface_normals.shape, face_centroids.shape)
                p = mp.plot(vertices, faces, colors, shading={"wireframe":True}, return_plot=True)
                #print(face_centroids, face_centroids+sf*mesh_normals)
                p.add_lines(face_centroids, face_centroids+sf*mesh_normals, shading={"line_color": "red", "line_thickness": 2.0})
                p.add_lines(face_centroids, face_centroids+sf*surface_normals, shading={"line_color": "green", "line_thickness": 2.0})
                #p.add_vectors(face_centroids, (0.5*mesh_normals), 0xff0000)
                #p.add_vectors(face_centroids, (0.5*surface_normals), 0x00ff00)
                    
#                     # Iterating over wires
#                     for wire_idx, wire in enumerate(topo.wires_from_face(face)):
#                         process_wire(wire_idx, wire)
                        
#                         # Iterating over edges
#                         for edge_idx, edge in enumerate(topo.edges_from_wire(wire)):
#                             process_edge(edge_idx, edge)
                            
#                             # Iterating over vertices
#                             for vert_idx, vert in enumerate(topo.vertices_from_edge(edge)):
#                                 process_vertex(vert_idx, vert)
                    
        


data/00000034.step [<class 'TopoDS_Solid'>, <class 'TopoDS_Solid'>, <class 'TopoDS_Solid'>]
Solid 0: ShapeType: 2, Closed: False, Orientable: False, Orientation: 0
Shell 0: ShapeType: 3, Closed: True, Orientable: True, Orientation: 0
Face 0: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 1: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 2: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 3: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 4: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 5: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(479.88153…

Solid 0: ShapeType: 2, Closed: False, Orientable: False, Orientation: 0
Shell 0: ShapeType: 3, Closed: True, Orientable: True, Orientation: 0
Face 0: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 1: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 2: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 3: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 4: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(99.840995…

Solid 0: ShapeType: 2, Closed: False, Orientable: False, Orientation: 0
Shell 0: ShapeType: 3, Closed: True, Orientable: True, Orientation: 0
Face 0: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 1: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 2: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 3: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 4: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 5: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 6: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 7: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 8: ShapeType: 4, Closed: False, Orientable: True, Orientation: 1
Face 9: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 10: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 11: ShapeType: 4, Closed: False, Orientable: True, Orientation: 0
Face 12: ShapeTy

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(18.340972…

In [78]:
def process_solid(solid_idx, solid):
    print("Solid %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(solid_idx, solid.ShapeType(), solid.Closed(), solid.Orientable(), solid.Orientation()))

def process_shell(shell_idx, shell):
    print("Shell %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(shell_idx, shell.ShapeType(), shell.Closed(), shell.Orientable(), shell.Orientation()))
    
def process_face(face_idx, face, first_vertex=0):
    print("Face %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(face_idx, face.ShapeType(), face.Closed(), face.Orientable(), face.Orientation()))
    surf = BRepAdaptor_Surface(face)
#     print(dir(surf.Surface()), dir(face))
#     stats["surfs"].append(surf_map[surf.GetType()])
#     d2_feat = convert_surface(surf)
#     #print(dir(face))
    face_orientation_wrt_surface_normal = face.Orientation()
    
    # Get mesh normals
    brep_tool = BRep_Tool()
    location = TopLoc_Location()
    mesh = brep_tool.Triangulation(face, location)
    verts = []
    tris = []
    normals = []
    surf_normals = []
    centroids = []
    if mesh != None:
        # Get vertices
        num_vertices = mesh.NbNodes()
        for i in range(1, num_vertices+1):
            verts.append(list(mesh.Node(i).Coord()))
        verts = np.array(verts)
       
        # Get faces
        num_tris = mesh.NbTriangles()
        for i in range(1, num_tris+1):
            index1, index2, index3 = mesh.Triangle(i).Get()
            if face_orientation_wrt_surface_normal == 0:
                tris.append([first_vertex + index1 - 1, first_vertex + index2 - 1, first_vertex + index3 - 1])
            elif face_orientation_wrt_surface_normal == 1:
                tris.append([first_vertex + index3 - 1, first_vertex + index2 - 1, first_vertex + index1 - 1])
            else:
                print("Broken face orientation", face_orientation_wrt_surface_normal)
            
            # Get mesh normals
            pt1 = verts[index1-1]
            pt2 = verts[index2-1]
            pt3 = verts[index3-1]
            centroid = (pt1 + pt2 + pt3)/3
            centroids.append(centroid)
            normal = np.cross(pt2-pt1, pt3-pt1)
            norm = np.linalg.norm(normal)
            if not np.isclose(norm, 0):
                normal /= norm
            if face_orientation_wrt_surface_normal == 1:
                normal = -normal
            normals.append(normal)
            
            # Get surface normals
            uv1 = convert_vec_to_np(mesh.UVNode(index1))
            uv2 = convert_vec_to_np(mesh.UVNode(index2))
            uv3 = convert_vec_to_np(mesh.UVNode(index3))
            #print(uv1, uv2, uv3)
            uvc = (uv1 + uv2 + uv3) / 3
            #print(uvc)
            uvc = gp_Pnt2d(uvc[0], uvc[1])
            surface_normal = get_surface_normal(uvc, surf)
            if face_orientation_wrt_surface_normal == 1:
                surface_normal = -surface_normal
            surf_normals.append(surface_normal)
    
    return verts.tolist(), tris, normals, centroids, surf_normals

def get_surface_normal(uv, surf):
    point = gp_Pnt()
    u_deriv_vec = gp_Vec()
    v_deriv_vec = gp_Vec()
    surf.D1(uv.X(), uv.Y(), point, u_deriv_vec, v_deriv_vec)
    u_deriv = np.array(list(u_deriv_vec.Coord()))
    v_deriv = np.array(list(v_deriv_vec.Coord()))
    face_normal = np.cross(u_deriv, v_deriv)
    norm = np.linalg.norm(face_normal)
    if not np.isclose(norm, 0):
        face_normal /= norm
    return face_normal

def convert_vec_to_np(vec):
    return np.array(list(vec.Coord()))

def process_wire(wire_idx, wire):
    print("Wire %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(wire_idx, wire.ShapeType(), wire.Closed(), wire.Orientable(), wire.Orientation()))
    #print(dir(wire))
    
def process_edge(edge_idx, edge):
    print("Edge %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(edge_idx, edge.ShapeType(), edge.Closed(), edge.Orientable(), edge.Orientation()))
    #print(dir(edge))
    
def process_vertex(vert_idx, vert):
    print("Vertex %i: ShapeType: %s, Closed: %s, Orientable: %s, Orientation: %s"%(vert_idx, vert.ShapeType(), vert.Closed(), vert.Orientable(), vert.Orientation()))
    #print(dir(vert))
    
edge_map = {0: "Line", 1: "Circle", 2: "Ellipse", 3: "Hyperbola", 4: "Parabola", 5: "Bezier", 6: "BSpline", 7: "Offset", 8: "Other"}
surf_map = {0: "Plane", 1: "Cylinder", 2: "Cone", 3: "Sphere", 4: "Torus", 5: "Bezier", 6: "BSpline", 7: "Revolution", 8: "Extrusion", 9: "Offset", 10: "Other"}

def process_part(part, convert=True):
    if convert:
        try:
            nurbs_converter = BRepBuilderAPI_NurbsConvert(part)
            nurbs_converter.Perform(part)
            part = nurbs_converter.Shape()
        except Exception as e:
            print("Conversion failed")
            print(e.args, str(e))
            return

    mesh = BRepMesh_IncrementalMesh(part, 0.95, False, 0.1, True)
    mesh.Perform()
    if not mesh.IsDone():
        print("Meshing failed.")
        return

    #topo_ex = TopologyExplorer(part)
    topo = Topo(part)
    return topo, mesh

In [55]:
import k3d


class Viewer():

    __plot = None
    __objects = {}
    __cnt = 0
    
    def __init__(self):
        # Plot settings
        settings = {"antialias":5, "axes": ["x", "y", "z"], "axes_helper": 1.0, "background_color":0xffffff,
           "camera_auto_fit": False, "grid_auto_fit": False, "grid_visible": False, "screenshot_scale": 2.0,
           "grid": (-1, -1, -1, 1, 1, 1), "lighting": 1.5, "menu_visibility": True, "voxel_paint_color": 0,
           "camera_fov": 60.0, "time": 0.0, "name": None, "camera_mode": 'trackball', "snapshot_include_js": True,
           "auto_rendering": True, "camera_no_zoom": False, "camera_no_rotate": False, "camera_no_pan": False,
           "fps": 25.0, "height": 1000} # manipulate_mode: translate, rotate, scale 
        self.__plot = k3d.plot(**settings)
        
    def __update_view(self):
        if len(self.__objects) == 0:
            return
        ma = np.zeros((len(self.__objects), 3))
        mi = np.zeros((len(self.__objects), 3))
        for r, obj in enumerate(self.__objects):
            ma[r] = self.__objects[obj]["max"]
            mi[r] = self.__objects[obj]["min"]
        ma = np.max(ma, axis=0)
        mi = np.min(mi, axis=0)
        diag = np.linalg.norm(ma-mi)
        mean = ((ma - mi) / 2 + mi).tolist()
        scale = 1.5 * diag
        camera = [mean[0], mean[1], mean[2]+scale, mean[0], mean[1], mean[2], 0.0, 1.0, 0.0]
        self.__plot.display()
        self.__plot.camera = camera
        self.__plot.camera_mode = "orbit"
        
    def add_mesh(self, vertices, faces, colors=0xffd200):
        # Convert to proper types
        vertices = vertices.astype("float32", copy=False)
        faces = faces.astype("uint32", copy=False)
        
        # Add to plot
        mesh = k3d.mesh(vertices, faces, color=colors, side="double")
        self.__plot += mesh
        mesh_obj = {}
        mesh_obj["max"] = np.max(vertices, axis=0)
        mesh_obj["min"] = np.min(vertices, axis=0)
        mesh_obj["mesh"] = mesh
        mesh_obj["type"] = "Mesh"
        self.__objects[self.__cnt] = mesh_obj
        self.__cnt += 1
        self.__update_view()
        
    def add_vectors(self, origins, vectors, colors=None):
        origins = origins.astype("float32", copy=False)
        vectors = vectors.astype("float32", copy=False)
        vectors = k3d.vectors(origins, vectors, color=colors)#, labels=[], label_size=1.5)
        self.__plot += vectors
        #self.__update_view()
        


def plot(vertices, faces, colors=0xffd200):
    plot = Viewer()
    plot.add_mesh(vertices, faces, colors)

    return plot
    

In [5]:
def read_step_file(filename, return_as_shapes=False, verbosity=False):
    assert os.path.isfile(filename)
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(filename)
    if status == IFSelect_RetDone:  # check status
        if verbosity:
            failsonly = False
            step_reader.PrintCheckLoad(failsonly, IFSelect_ItemsByEntity)
            step_reader.PrintCheckTransfer(failsonly, IFSelect_ItemsByEntity)
        shapes = []
        nr = 1
        try:
            while True:
                ok = step_reader.TransferRoot(nr)
                if not ok:
                    break
                _nbs = step_reader.NbShapes()
                shapes.append(step_reader.Shape(nr))  # a compound
                #assert not shape_to_return.IsNull()
                nr += 1
        except:
            print("No Shape", nr)
    else:
        raise AssertionError("Error: can't read file.")

    return shapes

In [80]:
def convert_curve(curve):
    d1_feat = {"type": edge_map[curve.GetType()]}
    c_type = d1_feat["type"]
    if c_type == "Line":
        c = curve.Line()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["direction"] = list(c.Direction().Coord())
        scale_factor = 1000.0
    elif c_type == "Circle":
        c = curve.Circle()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["z_axis"] = list(c.Axis().Direction().Coord())
        d1_feat["radius"] = c.Radius()
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        scale_factor = 1.0
    elif c_type == "Ellipse":
        c = curve.Ellipse()
        d1_feat["focus1"] = list(c.Focus1().Coord())
        d1_feat["focus2"] = list(c.Focus2().Coord())
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        d1_feat["z_axis"] = list(c.Axis().Direction().Coord())
        d1_feat["maj_radius"] = c.MajorRadius()
        d1_feat["min_radius"] = c.MinorRadius()
        scale_factor = 1.0
    elif c_type == "BSpline":
        c = curve.BSpline()
        c.SetNotPeriodic()
        d1_feat["rational"] = c.IsRational()
        d1_feat["closed"] = c.IsClosed()
        d1_feat["continuity"] = c.Continuity()
        d1_feat["degree"] = c.Degree()
        p = TColgp_Array1OfPnt(1, c.NbPoles())
        c.Poles(p)
        points = []
        for pi in range(p.Length()):
            points.append(list(p.Value(pi+1).Coord()))
        d1_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbPoles() + c.Degree() + 1)
        c.KnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d1_feat["knots"] = knots

        w = TColStd_Array1OfReal(1, c.NbPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.Length()):
            weights.append(w.Value(wi+1))
        d1_feat["weights"] = weights

        scale_factor = 1.0
    else:
        print("Unsupported type 3d", c_type)

    return d1_feat

def convert_2dcurve(curve, convert_2bs=False):
    curve_t = curve.Edge()
    d1_feat = {"type": edge_map[curve.GetType()], "interval": [curve.FirstParameter(), curve.LastParameter()]}        
    c_type = d1_feat["type"]

    if c_type == "Line":
        c = curve.Line()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["direction"] = list(c.Direction().Coord())
    elif c_type == "Circle":
        c = curve.Circle()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["radius"] = c.Radius()
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
    elif c_type == "Ellipse":
        c = curve.Ellipse()
        d1_feat["focus1"] = list(c.Focus1().Coord())
        d1_feat["focus2"] = list(c.Focus2().Coord())
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        d1_feat["maj_radius"] = c.MajorRadius()
        d1_feat["min_radius"] = c.MinorRadius()
    elif c_type == "BSpline":
        if not convert_2bs:
            c = curve.BSpline()
        else:
            c = curve
        c.SetNotPeriodic()
        d1_feat["rational"] = c.IsRational()
        d1_feat["closed"] = c.IsClosed()
        d1_feat["continuity"] = c.Continuity()
        d1_feat["degree"] = c.Degree()
        p = TColgp_Array1OfPnt2d(1, c.NbPoles())
        c.Poles(p)
        points = []
        for pi in range(p.Length()):
            points.append(list(p.Value(pi+1).Coord()))
        d1_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbPoles() + c.Degree() + 1)
        c.KnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d1_feat["knots"] = knots

        w = TColStd_Array1OfReal(1, c.NbPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.Length()):
            weights.append(w.Value(wi+1))
        d1_feat["weights"] = weights
    else:
        print("Unsupported type 2d", c_type)
    return d1_feat

def convert_surface(surf, convert_2bs=False):
    surf_t = surf.Face()
    d2_feat = {"type": surf_map[surf.GetType()]}

    s_type = d2_feat["type"]
        
    if s_type == "Plane":
        s = surf.Plane()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())

    elif s_type == "Cylinder":
        s = surf.Cylinder()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.Radius()

    elif s_type == "Cone":
        s = surf.Cone()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.RefRadius()
        d2_feat["angle"] = s.SemiAngle()
        d2_feat["apex"] = list(s.Apex().Coord())

    elif s_type == "Sphere":
        s = surf.Sphere()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.Radius()

    elif s_type == "Torus":
        s = surf.Torus()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["max_radius"] = s.MajorRadius()
        d2_feat["min_radius"] = s.MinorRadius()


    elif s_type == "Bezier":
        print("BEZIER SURF")

    elif s_type == "BSpline":
        if not convert_2bs:
            c = surf.BSpline()
        else:
            c = surf

        _round = lambda x: round(x, 15)
        d2_feat["trim_domain"] = list(map(_round, breptools_UVBounds(surf_t)))
        d2_feat["face_domain"] = list(map(_round, c.Bounds()))
        d2_feat["is_trimmed"] = d2_feat["trim_domain"] != d2_feat["face_domain"]
        #print(c.IsUPeriodic(), c.IsVPeriodic())
        c.SetUNotPeriodic()
        c.SetVNotPeriodic()
        #print(d2_feat["trim_domain"], d2_feat["face_domain"])
        d2_feat["u_rational"] = c.IsURational()
        d2_feat["v_rational"] = c.IsVRational()
        d2_feat["u_closed"] = c.IsUClosed()
        d2_feat["v_closed"] = c.IsVClosed()
        d2_feat["continuity"] = c.Continuity()
        d2_feat["u_degree"] = c.UDegree()
        d2_feat["v_degree"] = c.VDegree()

        p = TColgp_Array2OfPnt(1, c.NbUPoles(), 1, c.NbVPoles())
        c.Poles(p)
        points = []
        for pi in range(p.ColLength()):
            elems = []
            for pj in range(p.RowLength()):
                elems.append(list(p.Value(pi+1, pj+1).Coord()))
            points.append(elems)
        d2_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbUPoles() + c.UDegree() + 1)
        c.UKnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d2_feat["u_knots"] = knots

        k = TColStd_Array1OfReal(1, c.NbVPoles() + c.VDegree() + 1)
        c.VKnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d2_feat["v_knots"] = knots

        w = TColStd_Array2OfReal(1, c.NbUPoles(), 1, c.NbVPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.ColLength()):
            elems = []
            for wj in range(w.RowLength()):
                elems.append(w.Value(wi+1, wj+1))
            weights.append(elems)
        d2_feat["weights"] = weights

        scale_factor = 1.0

    elif s_type == "Revolution":
        s = surf.AxeOfRevolution()
        c = surf.BasisCurve()
        d1_feat = convert_curve(c)
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Direction().Coord())
        d2_feat["curve"] = d1_feat

    elif s_type == "Extrusion":
        c = surf.BasisCurve()
        d1_feat = convert_curve(c)
        d2_feat["direction"] = list(surf.Direction().Coord())
        d2_feat["curve"] = d1_feat
        
    else:
        print("Unsupported type", s_type)
    
    return d2_feat

In [6]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Core.BOPAlgo import BOPAlgo_Builder
def make_non_manifold_body():
        b1 = BRepPrimAPI_MakeBox(gp_Pnt(0, 0, 0), 10, 10, 10).Shape()
        b2 = BRepPrimAPI_MakeBox(gp_Pnt(5, 5, 5), 10, 10, 10).Shape()
        builder = BOPAlgo_Builder()
        builder.AddArgument(b1)
        builder.AddArgument(b2)
        builder.SetRunParallel(True)
        builder.Perform()
        #self.assertTrue(not builder.HasErrors())
        return builder.Shape()

In [20]:
b1 = make_non_manifold_body()

In [21]:
process_part(b1)